## Load Dataset

In [2]:
%load_ext autoreload
%autoreload 2

In [121]:
import pandas as pd
import numpy as np
from joblib import dump

In [122]:
df_raw =pd.read_csv('../data/raw/beer_reviews.csv')

In [196]:
df = df_raw.sample(n=5000, random_state=42)

In [197]:
df.shape

(5000, 13)

In [198]:
df_cleaned = df.copy()
df_cleaned.drop('brewery_id', axis=1, inplace=True)
df_cleaned.drop('beer_beerid', axis=1, inplace=True)
df_cleaned.drop('review_profilename', axis=1, inplace=True)
df_cleaned.dropna(inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)

In [199]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
num_cols = ['review_time', 'review_overall', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'beer_abv']
sc = StandardScaler()
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

In [200]:
cat_cols = ['brewery_name', 'beer_style', 'beer_name']
# creating instance of labelencoder
le = LabelEncoder()
X_cat = pd.DataFrame()

for col in cat_cols:
    X_cat[col] = le.fit_transform(df_cleaned[col])

X_cat.columns = cat_cols

In [ ]:
#OneHotEncoder
#Onehotencoder Target
y=

In [201]:
df_cleaned.drop(cat_cols, axis=1, inplace=True)

In [202]:
X = pd.concat([df_cleaned, X_cat], axis=1)
X.head()

review_time  review_overall  review_aroma  review_appearance  \
0     1.203123        0.227083      1.080284           1.031849   
1    -0.423751       -1.893147     -1.111049          -0.598979   
2    -0.040342        0.227083     -0.380605           0.216435   
3    -2.083433        0.933827      0.349839          -0.598979   
4    -0.730606        1.640570      0.349839           1.847264   

   review_palate  review_taste  beer_abv  brewery_name  beer_style  beer_name  
0      -0.403046     -0.436944  4.878019           598           4        332  
1      -1.143664     -1.820626 -0.609603           678          35       1891  
2      -0.403046     -0.436944 -0.864841           422          47         52  
3       0.337571      0.946737 -0.949921           173          36       1495  
4       1.818805      1.638578 -0.779762           918          78       2842

In [203]:
#set numpy=false
from src.data.sets import split_sets_random
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(X, target_col='beer_style', test_ratio=0.2, to_numpy=False)

In [204]:
#Print the dimensions of `X_train`, `X_val`, `X_test`
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(2877, 9)
(959, 9)
(960, 9)


In [205]:
#from src.data.sets import load_sets
#X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/')

In [206]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

## Baseline Model

In [244]:
from src.models.null import NullModel
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [245]:
from src.models.performance import print_class_perf
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.08828640945429267
F1 Training: 0.014324335995777922


## Define Architecture

In [254]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [263]:
from src.models.pytorch import PytorchMultiClass
model = PytorchMultiClass(X_train.shape[1])

In [264]:
from src.models.pytorch import get_device
device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=9, out_features=70, bias=True)
  (layer_out): Linear(in_features=70, out_features=150, bias=True)
  (softmax): Softmax(dim=1)
)

In [265]:
#print architecture
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=9, out_features=70, bias=True)
  (layer_out): Linear(in_features=70, out_features=150, bias=True)
  (softmax): Softmax(dim=1)
)


## Train model

In [266]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [267]:
N_EPOCHS = 50
BATCH_SIZE = 32

In [268]:
from src.models.pytorch import test_classification, train_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.1569	|	Acc: 0.6%
	(valid)	|	Loss: 0.1571	|	Acc: 0.0%
Epoch: 1
	(train)	|	Loss: 0.1570	|	Acc: 0.4%
	(valid)	|	Loss: 0.1560	|	Acc: 3.4%
Epoch: 2
	(train)	|	Loss: 0.1568	|	Acc: 1.1%
	(valid)	|	Loss: 0.1561	|	Acc: 3.2%
Epoch: 3
	(train)	|	Loss: 0.1566	|	Acc: 1.7%
	(valid)	|	Loss: 0.1566	|	Acc: 1.7%
Epoch: 4
	(train)	|	Loss: 0.1566	|	Acc: 1.6%
	(valid)	|	Loss: 0.1566	|	Acc: 1.7%
Epoch: 5
	(train)	|	Loss: 0.1566	|	Acc: 1.5%
	(valid)	|	Loss: 0.1566	|	Acc: 1.7%
Epoch: 6
	(train)	|	Loss: 0.1566	|	Acc: 1.5%
	(valid)	|	Loss: 0.1566	|	Acc: 1.7%
Epoch: 7
	(train)	|	Loss: 0.1566	|	Acc: 1.7%
	(valid)	|	Loss: 0.1566	|	Acc: 1.7%
Epoch: 8
	(train)	|	Loss: 0.1569	|	Acc: 0.7%
	(valid)	|	Loss: 0.1571	|	Acc: 0.0%
Epoch: 9
	(train)	|	Loss: 0.1570	|	Acc: 0.5%
	(valid)	|	Loss: 0.1571	|	Acc: 0.0%
Epoch: 10
	(train)	|	Loss: 0.1570	|	Acc: 0.4%
	(valid)	|	Loss: 0.1571	|	Acc: 0.0%
Epoch: 11
	(train)	|	Loss: 0.1570	|	Acc: 0.2%
	(valid)	|	Loss: 0.1570	|	Acc: 0.2%
Epoch: 12
	(train)	|	Loss:

In [185]:
torch.save(model, "../models/pytorch_1.pt")

## Test model

In [186]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.1569	|	Accuracy: 0.0


In [ ]:
#Hyperparameter Tuning

In [31]:
from skorch import NeuralNetClassifier

In [33]:
class MyClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        ...
 
    def forward(self, x):
        ...
        return x
 
# create the skorch wrapper
model = NeuralNetClassifier(
    module=MyClassifier
)

model = NeuralNetClassifier(
    module=MyClassifier,
    max_epochs=150,
    batch_size=10
)



In [35]:
class SonarClassifier(nn.Module):
    def __init__(self, n_layers=3):
        super().__init__()
        self.layers = []
        self.acts = []
        for i in range(n_layers):
            self.layers.append(nn.Linear(60, 60))
            self.acts.append(nn.ReLU())
            self.add_module(f"layer{i}", self.layers[-1])
            self.add_module(f"act{i}", self.acts[-1])
        self.output = nn.Linear(60, 1)
 
    def forward(self, x):
        for layer, act in zip(self.layers, self.acts):
            x = act(layer(x))
        x = self.output(x)
        return x
 
model = NeuralNetClassifier(
    module=SonarClassifier,
    max_epochs=150,
    batch_size=10,
    module__n_layers=2
)

In [37]:
print(model.initialize())

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=SonarClassifier(
    (layer0): Linear(in_features=60, out_features=60, bias=True)
    (act0): ReLU()
    (layer1): Linear(in_features=60, out_features=60, bias=True)
    (act1): ReLU()
    (output): Linear(in_features=60, out_features=1, bias=True)
  ),
)
